# Code to calculate FF5FM+M

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
currentPath = os.getcwd()
dataPath = (currentPath + '/../data/external')

In [3]:
# Read the two CRSP csv files. 
crspmret = pd.read_csv(dataPath + '/crspmret.csv')
crspminfo = pd.read_csv(dataPath + '/crspminfo.csv')

# Read the signals dataframes
df = pd.read_csv(dataPath+'/signed_predictors_all_wide.csv', usecols=['permno','yyyymm','BMdec'])

In [4]:
crsp = crspminfo.merge(crspmret, on=['permno','yyyymm']).drop('date',axis=1)
crsp = crsp.merge(df, on=['permno', 'yyyymm'])#.dropna(subset=['BMdec'])
crsp.dropna(subset=['BMdec'], inplace=True)

In [5]:
del df

In [11]:
199806 % 100

6

In [7]:
crsp['totalmelag'] = crsp.groupby('yyyymm')['melag'].transform('sum')

In [9]:
crsp.loc[crsp['yyyymm'] == 199901]

,permno,yyyymm,prc,exchcd,me,shrcd,siccd,me_nyse10,me_nyse20,ret,melag,BMdec,totalmelag
171,10001,199901,9.62500,3.0,2.330212e+04,11.0,4920.0,370368.0,454475.0,0.000000,23302.12500,0.770123,5.179262e+08
550,10002,199901,15.50000,3.0,1.197995e+05,11.0,6710.0,370368.0,454475.0,1.639344,117867.25000,0.437021,5.179262e+08
1837,10009,199901,22.00000,3.0,5.302000e+04,11.0,6020.0,370368.0,454475.0,4.761905,50610.00000,0.633649,5.179262e+08
2243,10012,199901,2.40625,3.0,4.297562e+04,10.0,3670.0,370368.0,454475.0,165.517247,16185.62500,0.094178,5.179262e+08
3138,10016,199901,16.43750,3.0,2.989488e+05,11.0,3560.0,370368.0,454475.0,-0.567108,300653.84375,0.397329,5.179262e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44750,10258,199901,2.09375,3.0,8.904509e+04,11.0,2830.0,370368.0,454475.0,19.642857,74425.75000,0.097862,5.179262e+08
45180,10259,199901,4.90625,3.0,7.651788e+04,11.0,3570.0,370368.0,454475.0,70.652175,41684.62500,0.424294,5.179262e+08
46662,10271,199901,38.00000,3.0,1.318600e+05,11.0,6710.0,370368.0,454475.0,-2.875399,135763.75000,0.610217,5.179262e+08
46930,10272,199901,20.00000,3.0,1.091900e+06,11.0,3820.0,370368.0,454475.0,20.754717,904229.68750,0.135380,5.179262e+08


In [8]:
crsp = crsp.drop(['siccd', 'me_nyse10','me_nyse20','melag'], axis=1)

In [9]:
# Selecting only the signal values in June
crspJune = crsp.loc[crsp['yyyymm']%100 == 6]

In [10]:
# Select shares with code 10, 11 and traded in NYSE (according to Fama and French ...)
temp = crspJune.loc[(crspJune['shrcd'].isin([10,11])) & (crspJune['exchcd'] == 1)]

In [11]:
temp

,permno,yyyymm,prc,exchcd,me,shrcd,ret,BMdec
1233,10006,195306,40.6250,1.0,26771.875,10.0,-10.103627,2.952443
1245,10006,195406,41.1250,1.0,29815.625,10.0,6.818182,3.477469
1257,10006,195506,66.5000,1.0,56990.500,10.0,5.346535,1.947503
1269,10006,195606,61.3750,1.0,77332.500,10.0,2.291667,1.008153
1281,10006,195706,60.0000,1.0,84720.000,10.0,-1.639344,1.304903
...,...,...,...,...,...,...,...,...
48131,10278,199306,18.0000,1.0,3340782.000,11.0,-7.974523,0.373774
48143,10278,199406,18.6250,1.0,3526308.500,11.0,1.904762,0.471837
48155,10278,199506,15.3750,1.0,2837241.000,11.0,1.540984,0.631355
48167,10278,199606,15.2500,1.0,2728103.000,11.0,-5.415384,0.602927


In [12]:
signalbreaks = temp.groupby('yyyymm')['BMdec'].apply(lambda x: x.quantile(q=[0.3,0.7])).reset_index().rename(columns={'level_1':'qlevel', 'BMdec': 'BMdec_breakpoint'})

In [13]:
sizebreak = temp.groupby('yyyymm')['me'].apply(lambda x: x.quantile(q=0.5)).reset_index().rename(columns={'me':'me_0.5'})

In [14]:
signalbreaks = signalbreaks.pivot(index='yyyymm', columns='qlevel', values='BMdec_breakpoint').rename(columns={0.3:'BMdec_0.3', 0.7:'BMdec_0.7'}).rename_axis(index='yyyymm', columns=None).reset_index()

In [15]:
signalbreaks

,yyyymm,BMdec_0.3,BMdec_0.7
0,195206,0.794888,0.905124
1,195306,0.742368,1.207412
2,195406,0.841143,1.449567
3,195506,0.602560,0.853812
4,195606,0.575893,0.793037
...,...,...,...
65,201706,0.206199,0.468776
66,201806,0.161151,0.392345
67,201906,0.189295,0.350929
68,202006,0.080621,0.153496


In [16]:
# Select shares with code 10,11 and exchange code 1,2 or 3
portfolio = crspJune.loc[(crspJune['exchcd'].isin([1,2,3])) & (crspJune['shrcd'].isin([10,11]))]

# Add signal breaks and size break to the portfolio df
portfolio = portfolio.merge(signalbreaks, on='yyyymm', how='left')
portfolio = portfolio.merge(sizebreak, on='yyyymm', how='left')

In [17]:
# Size Sort (Small and Big)
portfolio['FF_size_sort'] = np.select(
    [
        portfolio['me'] <= portfolio['me_0.5'],
        portfolio['me'] > portfolio['me_0.5']
    ],
    [
        'S',
        'B',
    ]
)

In [18]:
# Part of the HML sort (High, Medium, Low)
portfolio['BM_sort'] = np.select(
    [
        portfolio['BMdec'] <= portfolio['BMdec_0.3'],
        (portfolio['BMdec'] > portfolio['BMdec_0.3']) & (portfolio['BMdec'] <= portfolio['BMdec_0.7']),
        portfolio['BMdec'] > portfolio['BMdec_0.7']
    ],
    [
        portfolio['FF_size_sort']+'L',
        portfolio['FF_size_sort']+'M',
        portfolio['FF_size_sort']+'H'
    ]
)

In [19]:
portfolio['BM_sortlag'] = portfolio.groupby('permno')['BM_sort'].shift(1)
portfolio['melag'] = portfolio.groupby('permno')['me'].shift(1)

In [20]:
portfolio = portfolio.drop(['prc','me','ret','exchcd', 'shrcd', 'BMdec', 'BMdec_0.3', 'BMdec_0.7', 'me_0.5', 'FF_size_sort', 'BM_sort'], axis=1)

In [21]:
portfolio.loc[portfolio['yyyymm'] == 198906]

,permno,yyyymm,BM_sortlag,melag
1,10001,198906,SH,6200.00000
48,10005,198906,SH,916.12500
90,10010,198906,SL,17869.18750
104,10012,198906,NaN,NaN
138,10016,198906,SH,347820.00000
...,...,...,...,...
2299,10262,198906,SL,1565.71875
2335,10272,198906,SH,65444.25000
2358,10274,198906,NaN,NaN
2371,10279,198906,SM,253328.25000


In [22]:
crsp = crsp.merge(portfolio, on=['yyyymm','permno'], how='left')

In [23]:
crsp['year'] = np.floor(crsp['yyyymm']/100).astype(int)

In [24]:
(crsp['yyyymm']%100).min()

1

In [25]:
crsp['adj_year'] = np.select(
    [
        crsp['yyyymm']%100 >= 6,
        crsp['yyyymm']%100 < 6
    ],
    [
        np.floor(crsp['yyyymm']/100).astype(int),
        (np.floor(crsp['yyyymm']/100)-1).astype(int)
    ]
)

In [26]:
crsp['BM_filled'] = crsp.groupby(['permno', 'adj_year'])['BM_sortlag'].ffill()
crsp['melag_filled'] = crsp.groupby(['permno', 'adj_year'])['melag'].ffill()

In [27]:
crsp.loc[(crsp['adj_year'] == 1968) & (crsp['permno'] == 10268)]

,permno,yyyymm,prc,exchcd,me,shrcd,ret,BMdec,BM_sortlag,melag,year,adj_year,BM_filled,melag_filled
27678,10268,196806,18.875,1.0,10419.000,10.0,1.328947,1.134514,SH,6048.0,1968,1968,SH,6048.0
27679,10268,196807,18.125,1.0,10005.000,10.0,-3.973510,1.134514,NaN,NaN,1968,1968,SH,6048.0
27680,10268,196808,18.750,1.0,10350.000,10.0,3.448276,1.134514,NaN,NaN,1968,1968,SH,6048.0
27681,10268,196809,19.625,1.0,11048.875,10.0,6.760000,1.134514,NaN,NaN,1968,1968,SH,6048.0
27682,10268,196810,18.125,1.0,10204.375,10.0,-7.643312,1.134514,NaN,NaN,1968,1968,SH,6048.0
27683,10268,196811,21.375,1.0,12034.125,10.0,17.931035,1.134514,NaN,NaN,1968,1968,SH,6048.0
27684,10268,196812,22.250,1.0,12571.250,10.0,6.175439,1.134514,NaN,NaN,1968,1968,SH,6048.0
27685,10268,196901,19.250,1.0,10876.250,10.0,-13.483146,1.134514,NaN,NaN,1969,1968,SH,6048.0
27686,10268,196902,17.875,1.0,10099.375,10.0,-7.142857,1.134514,NaN,NaN,1969,1968,SH,6048.0
27687,10268,196903,17.250,1.0,9936.000,10.0,-1.566434,1.134514,NaN,NaN,1969,1968,SH,6048.0


In [28]:
factor_portfolio = crsp[['permno', 'yyyymm', 'ret','BM_filled','melag_filled','adj_year']]

In [29]:
factor_portfolio

,permno,yyyymm,ret,BM_filled,melag_filled,adj_year
0,10001,198712,-3.353535,NaN,NaN,1987
1,10001,198805,1.980198,NaN,NaN,1987
2,10001,198806,-1.203884,NaN,NaN,1988
3,10001,198807,3.000000,NaN,NaN,1988
4,10001,198808,2.912621,NaN,NaN,1988
...,...,...,...,...,...,...
28515,10281,199101,-1.428571,SL,11518.4375,1990
28516,10281,199102,-8.695652,SL,11518.4375,1990
28517,10281,199103,6.349207,SL,11518.4375,1990
28518,10281,199104,10.447761,SL,11518.4375,1990


In [30]:
factor_portfolio.loc[factor_portfolio['yyyymm'] == 195306]

,permno,yyyymm,ret,BM_filled,melag_filled,adj_year
594,10006,195306,-10.103627,NaN,NaN,1953
9445,10102,195306,-8.796296,SH,69084.0,1953
14011,10145,195306,0.000000,BL,663093.0,1953
15540,10153,195306,-5.555556,BH,133642.5,1953
16724,10161,195306,0.636943,SM,78300.0,1953
19326,10188,195306,-8.974359,SL,17017.0,1953
22232,10225,195306,2.599653,BL,374332.0,1953
23420,10233,195306,-1.680672,SM,44550.0,1953
24531,10241,195306,0.361011,BM,372845.5,1953
27498,10268,195306,0.800000,SH,2964.0,1953


In [31]:
# Should I check for stocks that are traded for the entirety of the year?
factor_portfolio = factor_portfolio.dropna()

In [32]:
factor_test = factor_portfolio.groupby(['yyyymm','BM_filled']).apply(lambda x: np.sum(x['ret']*(x['melag_filled']/x['melag_filled'].sum()))).reset_index()

In [33]:
factor_test

,yyyymm,BM_filled,0
0,195306,BH,-5.555556
1,195306,BL,0.938028
2,195306,BM,0.361011
3,195306,SH,-8.401512
4,195306,SL,-8.974359
...,...,...,...
4285,202111,SL,-14.951638
4286,202112,BH,1.722718
4287,202112,BM,1.175626
4288,202112,SH,8.755012


In [34]:
factor_test = factor_test.pivot(index='yyyymm', columns='BM_filled', values=0)


In [35]:
# Check if dropping NaNs is correct
factor_test = factor_test.dropna()

In [36]:
factor_test

BM_filled,BH,BL,BM,SH,SL,SM
yyyymm,,,,,,
195306,-5.555556,0.938028,0.361011,-8.401512,-8.974359,-0.203511
195307,-2.000000,2.771906,6.043166,0.519382,-7.042254,0.403394
195308,-8.928572,-3.166324,-8.561644,-5.166337,-21.212122,-2.779255
195309,-1.960784,0.738001,7.116105,-1.349213,13.461539,1.325817
195310,0.000000,0.104207,5.524475,1.776441,-3.389831,8.568776
...,...,...,...,...,...,...
200501,-18.038252,-1.001094,-11.360814,-4.366504,-10.490103,-8.700897
200502,-3.143499,-3.561879,-3.662578,-4.090776,5.617583,-4.184484
200503,-4.753249,-2.986358,-3.680832,-0.383988,-5.530296,-1.379234


In [37]:
HML = 0.5*(factor_test['SH']+factor_test['BH']) - 0.5*(factor_test['SL']+factor_test['BL'])

In [117]:
HML.mean()

-0.8323443234786131

In [43]:
HML = HML.reset_index()
HML.loc[HML['yyyymm'] == 199601]

,index,yyyymm,0
187,187,199601,1.560832


,yyyymm,0
0,195306,-2.960368
1,195307,1.394865
2,195308,5.141768
3,195309,-8.754769
4,195310,2.531032
...,...,...
281,200501,-5.456779
282,200502,-4.644990
283,200503,1.689708
284,200504,7.952455
